# Topic Modelling 

### Importing libraries

In [62]:
import nltk; nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vibha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Vibha\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [82]:

import numpy as np 
import pandas as pd
from pprint import pprint

# spacy for lemmatization
import spacy

# gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


In [64]:
! pip install pyLDAvis

In [65]:
# For visualization
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

#### Preparing stop words

In [66]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Dataset is obtained from kaggle https://www.kaggle.com/datasets/yasserh/amazon-product-reviews-dataset

In [67]:
# Loading the dataset.
data = pd.read_csv('amazon.csv')

In [68]:
data.shape

(1597, 27)

In [69]:
data.head(2)

,id,asins,brand,categories,colors,dateAdded,dateUpdated,dimension,ean,keys,...,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,sizes,upc,weight
0,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,5.0,https://www.amazon.com/Kindle-Paperwhite-High-...,I initially had trouble deciding between the p...,"Paperwhite voyage, no regrets!",NaN,NaN,Cristina M,NaN,NaN,205 grams
1,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,5.0,https://www.amazon.com/Kindle-Paperwhite-High-...,Allow me to preface this with a little history...,One Simply Could Not Ask For More,NaN,NaN,Ricky,NaN,NaN,205 grams


In [70]:
data.groupby(['id', 'reviews.text']).size()


id                    reviews.text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [71]:
data = pd.DataFrame(data)

###  Data pre-processing
Here I am using tokenization to split the raw text into small chunks of words or sentences called as tokens.

In [72]:

# Convert to list
data_1 = data['reviews.text'].values.tolist()


In [73]:
print(data_1[0][0:90])

I initially had trouble deciding between the paperwhite and the voyage because reviews mor


### Performing lemmatization

In [74]:
def lemmatization(texts, allowed_postags=['NOUN','ADJ','VERB','ADV']):
    nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
    texts_out=[]
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return(texts_out)
lemmatized_txt = lemmatization(data_1)
print(lemmatized_txt[0][:90])

initially trouble decide paperwhite voyage review more less say same thing paperwhite grea


In [75]:
def sentences_to_words(texts):
    for sentence in texts:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
    return(sentence)
data_words = list(sentences_to_words(lemmatized_txt))
print(data_words[0][:90])
print(data_words[:1])

['initially', 'trouble', 'decide', 'paperwhite', 'voyage', 'review', 'more', 'less', 'say', 'same', 'thing', 'paperwhite', 'great', 'spend', 'money', 'go', 'voyage', 'fortunately', 'friend', 'own', 'so', 'end', 'buy', 'paperwhite', 'basis', 'model', 'now', 'ppi', 'dollar', 'jump', 'turn', 'pricey', 'voyage', 'page', 'press', 'always', 'sensitive', 'fine', 'specific', 'setting', 'need', 'auto', 'light', 'week', 'love', 'paperwhite', 'regret', 'touch', 'screen', 'receptive', 'easy', 'use', 'keep', 'light', 'specific', 'setting', 'regardless', 'time', 'day', 'case', 'hard', 'change', 'setting', 'either', 'only', 'change', 'light', 'level', 'certain', 'time', 'day', 'now', 'then', 'glad', 'go', 'international', 'shipping', 'option', 'extra', 'expense', 'delivery', 'time', 'tracking', 'need', 'worry', 'custom', 'use', 'third', 'party', 'shipping']
[['initially', 'trouble', 'decide', 'paperwhite', 'voyage', 'review', 'more', 'less', 'say', 'same', 'thing', 'paperwhite', 'great', 'spend', 'mo

Now here I have the stop words removed and have a cleaner text.

### Dictionary to look up words and their frequency

In [76]:
id_to_word = corpora.Dictionary(data_words)
# Create Corpus
new = data_words

# Term Document Frequency
corpus = [id_to_word.doc2bow(text) for text in new]
print(corpus[0][:20])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)]


In [77]:
from collections import Counter
bow_simple = Counter(id_to_word)
# Print the 10 most common tokens
print(bow_simple.most_common(10))

[(660, 'zw'), (3075, 'zooming'), (3320, 'zoom'), (3713, 'zombie'), (1974, 'zippy'), (962, 'zip'), (3418, 'zen'), (3282, 'yr'), (3660, 'ypithvtylczt'), (1312, 'youtube')]


### Building a topic model

In [78]:
LDA_model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id_to_word,
                                           num_topics=20, random_state=100,
                                           update_every=1, chunksize=100,
                                           passes=10, alpha='auto',
                                           per_word_topics=True)

In [79]:
pprint(LDA_model.print_topics())
doc_LDA = LDA_model[corpus]

[(0,
  '0.106*"tablet" + 0.063*"model" + 0.053*"complete" + 0.050*"screen" + '
  '0.045*"first" + 0.038*"new" + 0.035*"touch" + 0.030*"early" + '
  '0.027*"consider" + 0.026*"figure"'),
 (1,
  '0.087*"suppose" + 0.072*"music" + 0.047*"exactly" + 0.045*"average" + '
  '0.044*"free" + 0.043*"travel" + 0.043*"safe" + 0.041*"mean" + 0.023*"more" '
  '+ 0.021*"so"'),
 (2,
  '0.040*"work" + 0.038*"just" + 0.037*"use" + 0.036*"design" + 0.033*"nice" + '
  '0.033*"set" + 0.029*"really" + 0.028*"so" + 0.026*"love" + 0.024*"buy"'),
 (3,
  '0.051*"read" + 0.039*"light" + 0.035*"easily" + 0.033*"way" + 0.031*"so" + '
  '0.028*"finish" + 0.028*"book" + 0.027*"line" + 0.023*"turn" + 0.021*"look"'),
 (4,
  '0.046*"sound" + 0.044*"well" + 0.041*"feel" + 0.033*"more" + '
  '0.029*"product" + 0.027*"review" + 0.027*"people" + 0.026*"year" + '
  '0.025*"think" + 0.025*"purchase"'),
 (5,
  '0.117*"question" + 0.105*"ask" + 0.083*"answer" + 0.076*"replace" + '
  '0.047*"drop" + 0.046*"month" + 0.042*"clean

### Compute model and coherence score
to judge how good the given topic model is 

In [80]:
# Coherence score 
lda_coherence_model = CoherenceModel(model=LDA_model, texts=data_words, dictionary=id_to_word, coherence='c_v')
coherence_lda= lda_coherence_model.get_coherence()
print("Coherence score: ", coherence_lda)

Coherence score:  0.4096251425298079


### Visualization

In [81]:
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(LDA_model, corpus, id_to_word,mds='mmds')
vis

C:\Users\Vibha\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.337248 -0.174339       1        1  14.671831
4     -0.449285 -0.012235       2        1  14.495985
2     -0.294811 -0.347688       3        1  13.786763
3     -0.116981 -0.370938       4        1   9.602241
8     -0.130863 -0.012863       5        1   6.170067
17    -0.186526  0.209094       6        1   6.067151
18    -0.305205  0.329334       7        1   5.470172
11    -0.080172 -0.176404       8        1   5.336462
1      0.046629 -0.402160       9        1   3.817075
0      0.352790 -0.250549      10        1   3.441762
6      0.227063 -0.309253      11        1   2.986732
15    -0.042608  0.497514      12        1   2.928118
16    -0.363786  0.125565      13        1   2.839497
13     0.289901  0.369390      14        1   2.194500
7      0.124104  0.146907      15        1   1.694168
19     0.069129  0.314736      16        1   1.416205
10     0.160517 -0.062122      17        1   1.100291
14     0.416556 -0.056338      18        1   0.992383
5      0.380240  0.131064      19        1   0.780502
12     0.240555  0.051284      20        1   0.208096, topic_info=           Term         Freq        Total Category  logprob  loglift
797   headphone  1404.000000  1404.000000  Default  30.0000  30.0000
860        fall   670.000000   670.000000  Default  29.0000  29.0000
237        work  1001.000000  1001.000000  Default  28.0000  28.0000
989       sound  1471.000000  1471.000000  Default  27.0000  27.0000
634      tablet   934.000000   934.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
1844     gaming     0.041338     1.718183  Topic20  -8.7124   2.4477
143        year     1.540460  1083.688520  Topic20  -5.0944  -0.3811
1916      oasis     0.041338     1.718278  Topic20  -8.7124   2.4476
1223    command     0.041338     1.718359  Topic20  -8.7124   2.4476
1216    speaker     0.041338     1.718960  Topic20  -8.7124   2.4472

[972 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
783       1  0.978867  ability
406       1  0.156295     able
406       4  0.027993     able
406       7  0.408233     able
406       8  0.086312     able
...     ...       ...      ...
143      20  0.001846     year
151       4  0.390872      yet
151      13  0.201020      yet
151      15  0.122846      yet
151      16  0.268027      yet

[1265 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 5, 3, 4, 9, 18, 19, 12, 2, 1, 7, 16, 17, 14, 8, 20, 11, 15, 6, 13])